# Setup

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

In [ ]:
import os
project_folder = '/gdrive/MyDrive/ProjectCIRI'
my_module_path = os.path.join(project_folder, 'code', 'ciri_utils')

In [ ]:
!pip install $my_module_path

Processing /gdrive/MyDrive/ProjectCIRI/code/ciri_utils
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 797.1 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 61.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.1/65.1 MB 8.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-

In [ ]:
import ray

from ray import tune
from ciri_utils.engine_v2 import CIRI_trainer

In [ ]:
# Paths for the dataset
root_base = os.path.join(project_folder, 'Incidents-subset')
root_augmented = os.path.join(project_folder, 'augmented_images')

data_folders=[root_base, root_augmented]
persistence_path = os.path.join(project_folder, 'checkpoints', 'HPT_resnet_50')

# Preparation

In [ ]:
selected_model="resnet50"

In [ ]:
ciri_trainer = CIRI_trainer(model=selected_model,
                            data_folders=data_folders,
                            data_prop=0.8,
                            sample_indices=0.2)

# Hyperparameter tuning

Perform hyper parameter tuning with nested cross-validation on a 20% sample of the dataset.

In [ ]:
search_space={
	'epochs': tune.choice([5, 10, 20]),
	'batch_size': tune.choice([32, 64]),
	'lr': tune.loguniform(1e-4, 1e-2)
}

In [ ]:
hp_nest_cv = ciri_trainer.cross_validate(
    run_name="resnet50_hpt",
    config=search_space,
    outer_cv_k=5,
    inner_cv_k=3,
    tune_hyperparams=True,
    num_samples=5,
    results_persist_dir=persistence_path
)

Output hidden; open in https://colab.research.google.com to view.

## Examine results

In [ ]:
import json
import re
import pandas as pd

from pprint import pprint

In [ ]:
best_experiments = []
for i in range(5):
  for j in range(3):
    file_summary = f"hpt_{selected_model}_hpt_outer_{i}_inner_{j}_exp_summary.csv"
    file_path = os.path.join(persistence_path, file_summary)
    tmp_df = pd.read_csv(file_path, usecols=["loss", "accuracy", "trial_id", "time_total_s"])
    tmp_df = tmp_df.loc[tmp_df['accuracy'] == tmp_df['accuracy'].max()]
    tmp_df['outer_fold'] = i
    tmp_df['inner_fold'] = j
    best_experiments.append(
        tmp_df
    )

best_experiments = pd.concat(best_experiments)
best_experiments = best_experiments.sort_values(by=['accuracy', 'loss', 'time_total_s'], ascending=[False, True, True])
best_experiments

,loss,accuracy,trial_id,time_total_s,outer_fold,inner_fold
0,2.839263,0.391576,6df97_00000,1531.414467,1,0
3,2.251570,0.379095,5d361_00003,1407.880165,2,1
0,2.042714,0.357812,04ddc_00000,709.957051,0,2
0,3.019495,0.350000,e8be0_00000,1515.005401,3,2
2,1.960826,0.329173,ccff1_00002,346.073121,0,0
0,2.905614,0.326562,a9418_00000,1386.689075,0,1
3,2.117947,0.319813,340bf_00003,293.687831,2,0
0,2.061533,0.267188,fc155_00000,1462.282066,1,2
0,2.319158,0.265625,51d1e_00000,360.897560,1,1
0,2.124572,0.254290,50b8f_00000,1580.006551,3,1


In [ ]:
#best_experiments.to_csv(os.path.join(persistence_path, f"HP_RANKING.{selected_model}_overview.csv"), index=False)

In [ ]:
best_params_path = os.path.join(persistence_path,
                                f"{selected_model}_hpt_outer_{best_experiments.iloc[0].outer_fold}_inner_{best_experiments.iloc[0].inner_fold}",
                                "params.json")
with open(best_params_path, 'r') as params_file:
  best_params = json.load(params_file)

best_params = {k: v for k, v in best_params['train_loop_config'].items() if k in search_space.keys()}

pprint(best_params)


{'batch_size': 32, 'epochs': 20, 'lr': 0.0006769998458972315}


# Cross-validation on entire dataset

In [ ]:
ciri_trainer = CIRI_trainer(model=selected_model,
                            data_folders=data_folders,
                            data_prop=0.8)

In [ ]:
persistence_path = os.path.join(project_folder, 'checkpoints', f'CV_{selected_model}')
os.makedirs(persistence_path, exist_ok=True)

In [ ]:
cv_whole = ciri_trainer.cross_validate(
    run_name="resnet50_cv",
    config={
        **best_params,
        "additional_metrics": ['precision', 'recall', 'f1', 'confusion_matrix']
    },
    outer_cv_k=5,
    inner_cv_k=0,
    tune_hyperparams=False,
    results_persist_dir=persistence_path
)

Output hidden; open in https://colab.research.google.com to view.

# Transfer learning

In [ ]:
ciri_trainer = CIRI_trainer(model=selected_model,
                            data_folders=data_folders,
                            data_prop=0.8)

In [ ]:
# project_folder = ... # Redefine here project folder if necessary
persistence_path = os.path.join(project_folder, 'checkpoints', f'TransferLearning_{selected_model}')
os.makedirs(persistence_path, exist_ok=True)

In [ ]:
tl_results = ciri_trainer.train(
    run_name=f'{selected_model}_transfer_learning_uftest',
    config={
        **{'batch_size': 32, 'epochs': 5, 'lr': 0.0006769998458972315},
        'additional_metrics': ['precision', 'recall', 'f1', 'confusion_matrix'],
        'weights': 'IMAGENET1K_V2',
        'unfreeze': ['layer4']
    },
    persist_dir=persistence_path
)

/usr/lib/python3.10/subprocess.py:1796: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = _posixsubprocess.fork_exec(
2024-04-17 14:39:09,738	INFO worker.py:1752 -- Started a local Ray instance.
2024-04-17 14:39:11,813	INFO tune.py:263 -- Initializing Ray automatically. For cluster usage or custom Ray initialization, call `ray.init(...)` before `<FrameworkTrainer>(...)`.
2024-04-17 14:39:11,825	INFO tune.py:622 -- [output] This will use the new output engine with verbosity 1. To disable the new output and use the legacy output engine, set the environment variable RAY_AIR_NEW_OUTPUT=0. For more information, please see https://github.com/ray-project/ray/issues/36949



View detailed results here: /root/ray_results/resnet50_transfer_learning_uftest
To visualize your results with TensorBoard, run: `tensorboard --logdir /tmp/ray/session_2024-04-17_14-39-05_869552_215/artifacts/2024-04-17_14-39-11/resnet50_transfer_learning_uftest/driver_artifacts`


(TrainTrainable pid=1649) 2024-04-17 14:39:35.588215: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
(TrainTrainable pid=1649) 2024-04-17 14:39:35.588678: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
(TrainTrainable pid=1649) 2024-04-17 14:39:35.590545: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
(TrainTrainable pid=1649) 2024-04-17 14:39:37.663906: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT



Training started with configuration:
+--------------------------------------------------------------+
| Training config                                              |
+--------------------------------------------------------------+
| train_loop_config/additional_metrics    ...onfusion_matrix'] |
| train_loop_config/batch_size                              32 |
| train_loop_config/data_folders          ...ugmented_images'] |
| train_loop_config/data_prop                              0.8 |
| train_loop_config/epochs                                   5 |
| train_loop_config/lr                   0.0006769998458972315 |
| train_loop_config/model                             resnet50 |
| train_loop_config/sample_indices                             |
| train_loop_config/unfreeze                        ['layer4'] |
| train_loop_config/weights                      IMAGENET1K_V2 |
+--------------------------------------------------------------+


(TorchTrainer pid=1649) Started distributed worker processes: 
(TorchTrainer pid=1649) - (ip=172.28.0.12, pid=1802) world_rank=0, local_rank=0, node_rank=0
(RayTrainWorker pid=1802) Setting up process group for: env:// [rank=0, world_size=1]
(RayTrainWorker pid=1802) [W Utils.hpp:133] Warning: Environment variable NCCL_ASYNC_ERROR_HANDLING is deprecated; use TORCH_NCCL_ASYNC_ERROR_HANDLING instead (function getCvarInt)
(RayTrainWorker pid=1802) 2024-04-17 14:39:44.232690: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
(RayTrainWorker pid=1802) 2024-04-17 14:39:44.232769: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
(RayTrainWorker pid=1802) 2024-04-17 14:39:44.234160: E external/local_xla/xla/stream_executor/cuda/cuda

...

Epoch (training) 2:  66%|██████▋   | 200/301 [05:02<02:32,  1.51s/it]
(RayTrainWorker pid=1802) /usr/local/lib/python3.10/dist-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
(RayTrainWorker pid=1802)   warnings.warn(
Epoch (test) 2: 100%|██████████| 76/76 [01:45<00:00,  1.39s/it]
(RayTrainWorker pid=1802) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/root/ray_results/resnet50_transfer_learning_uftest/TorchTrainer_418ea_00000_0_2024-04-17_14-39-12/checkpoint_000001)
Epoch (training) 3:   0%|          | 0/301 [00:00<?, ?it/s]



Training finished iteration 2 at 2024-04-17 16:36:20. Total running time: 1hr 57min 7s
+---------------------------------------------+
| Training result                             |
+---------------------------------------------+
| checkpoint_dir_name       checkpoint_000001 |
| time_this_iter_s                  553.63217 |
| time_total_s                      7001.4291 |
| training_iteration                        2 |
| accuracy                             0.8975 |
| confusion_matrix       ..., 1, 2, 9, 9, 7]) |
| f1                                  0.89754 |
| loss                                0.34293 |
| precision                           0.90037 |
| recall                               0.8975 |
| summary/epoch/0                         1.0 |
| summary/epoch/1                         2.0 |
| summary/train_acc/0      0.7905208333333333 |
| summary/train_acc/1      0.9183333333333333 |
| summary/train_loss/0     0.6672585769448169 |
| summary/train_loss/1    0.25891431158662237 |


Epoch (training) 3:  89%|████████▉ | 268/301 [06:34<00:49,  1.51s/it]
(RayTrainWorker pid=1802) /usr/local/lib/python3.10/dist-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
(RayTrainWorker pid=1802)   warnings.warn(
Epoch (test) 3: 100%|██████████| 76/76 [01:46<00:00,  1.40s/it]
(RayTrainWorker pid=1802) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/root/ray_results/resnet50_transfer_learning_uftest/TorchTrainer_418ea_00000_0_2024-04-17_14-39-12/checkpoint_000002)
Epoch (training) 4:   0%|          | 0/301 [00:00<?, ?it/s]



Training finished iteration 3 at 2024-04-17 16:45:30. Total running time: 2hr 6min 17s
+---------------------------------------------+
| Training result                             |
+---------------------------------------------+
| checkpoint_dir_name       checkpoint_000002 |
| time_this_iter_s                  549.81427 |
| time_total_s                     7551.24338 |
| training_iteration                        3 |
| accuracy                            0.87833 |
| confusion_matrix       ..., 1, 2, 9, 9, 7]) |
| f1                                  0.87997 |
| loss                                 0.4179 |
| precision                           0.88834 |
| recall                              0.87833 |
| summary/epoch/0                         1.0 |
| summary/epoch/1                         2.0 |
| summary/epoch/2                         3.0 |
| summary/train_acc/0      0.7905208333333333 |
| summary/train_acc/1      0.9183333333333333 |
| summary/train_acc/2      0.9573958333333333 |


Epoch (training) 4:  28%|██▊       | 84/301 [02:06<05:35,  1.54s/it]
